Simple Neural Network Setup
1. Input layer (#features for each input) - 7 for seeds dataset
2. Hidden layer(#input_layer_neurons + 1 for bias) - Assume 4 for now
3. Output layer(#hidden_layer_neurons + 1 for bias) - 3 for 3 output classes

In [1]:
from random import seed
from random import random
from math import exp
import pprint as pp

Weights:
1. Each neuron in hidden layer should consist one weight for each input(feature) + one weight for bias
2. Here we have 7 features, so each hidden_layer_neuron should have (7+1 = 8) weights
3. Since we have 4 hidden_layer_neurons we need to have 4*8 weights

hidden_layer below is list of 4 weights, one for each hidden neuron of hidden layer

Lets do the same thing for output layer too. output_layer has 3 neurons(one for each kind of seed). Each of the output neuron gets input from all 4 hidden layer neurons. So output_layer is a list of 3 dictionaries and each dictionary holds a list of 4 weights.

Lets combine above steps into a function

In [2]:
def buildNetwork(n_input, n_hidden, n_output):
    network = []
#     print(n_input, n_hidden)
    hidden_layer = [{'weights':[random() for i in range(n_input+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden+1)]} for i in range(n_output)]
    network.append(output_layer)
    return network

In [3]:
seed(10)
# networkSeeds = buildNetwork(7,4,3)

# print('Total number of layers in network(excluding input layer):',len(networkSeeds))
# print('Number of hidden layers:', len(networkSeeds[0]))
# # print(networkTest[0])
# print('Number of output layers:', len(networkSeeds[1]))
# # print(networkTest[1])
# pp.pprint(networkSeeds)

Now that we have all pieces in place, its time to do some calculations.
1. Neuron's output: Each neuron output is the is the dot product of its inputs and weights and bias added to the dot product
    - @each_neuron_output: (for all inputs input[i] * weights[i])+bias.
    - Each of the input's feature needs to be converted into float as it is stored as string in the list/file.

In [4]:
def calculateOutput(input_features, layer_n):
    output = []
    product = 0
    bias = 0
    for eachNeuron in layer_n:
        wlist = eachNeuron['weights']
        bias = wlist[-1]
        for i in range(len(input_features)):
            product = product + float(input_features[i]) * wlist[i]
        eachNeuron['output'] = activation((product + bias))
#         pp.pprint(eachNeuron)
        output.append(activation((product + bias)))
#     pp.pprint(layer_n)
    return output

# test_output_hidden = calculateOutput([1.12,2.23,3.34,4.45,5.56,6.67,7.78], networkSeeds[0])
# test_output_final = calculateOutput(test_output_hidden, networkSeeds[1])

# print('Output of hidden layer prior to activation:',test_output_hidden)
# print('Output of final layer prior to activation:',test_output_final)

Now the neuron_output is not passed on to the next layer as it is. An activation function is applied to the neuron_output and that result is passed to the next layer. (My understanding as to why I need activation function is foggy at that moment, need to explore that)

Here I use sigmoid activation among many other options.
    - sigmoid_activation = 1/(1 + e(-neuron_output))

In [5]:
def sigmoidActivation(layer_output):
    activatedOutput = []
    for eachNeuronOutput in layer_output:
        activatedOutput.append(1/(1 + exp(-eachNeuronOutput)))
    return activatedOutput

def activation(output):
    activation = 1/(1 + exp(-output))
    return activation


# activatedOutputHidden = sigmoidActivation(test_output_hidden)
# activatedOutputFinal = sigmoidActivation(test_output_final)
# print('Activated output of hidden layer:', activatedOutputHidden)
# print('Activated output of final layer:', activatedOutputFinal)

Forward propagation: Calculate output of each layer and provide it as input to next layer untill final output layer is reached. 

In [6]:
import csv
with open('wheat-seeds.csv', 'r') as f:
    reader = csv.reader(f)
    input_seeds_list = list(reader)
# input_seeds_list = [float(i) for i in input_seeds_list]
temp_input_list = []
expected = []
for eachList in input_seeds_list:
    expected.append(eachList[-1])
    del eachList[-1]
    for i in range(len(eachList)):
        eachList[i] = float(eachList[i])
    temp_input_list.append(eachList)
input_seeds_list = temp_input_list 

Make the expected output into one-hot vector

In [7]:
expected_list = []
max_class = max(expected)
min_class = min(expected)
# print(min_class)
# print(max_class)
for eachClass in expected:
    exp_vec = [0,0,0]
    for i in range(int(max_class)):
        if i == int(eachClass) - 1:
            exp_vec[i] = 1
        else:
            exp_vec[i] = 0
    expected_list.append(exp_vec)
print(expected_list[-1])

[0, 0, 1]


In [8]:
network = buildNetwork(2, 1, 2)

In [12]:

# def forwardPropagation(input_seeds_list,network, n_hidden, n_output):
#     n_input = len(input_seeds_list[0])
# #     myNetwork = buildNetwork(n_input, n_hidden, n_output)
#     predictions = []
# #     print(len(input_seeds_list))

#     for i in range(len(input_seeds_list)):
#         inputSeed = input_seeds_list[i]
#         #Hidden layer calculations:
#         hiddenLayerOutput = calculateOutput(inputSeed, network[0])
#         hiddenLayerOutput = sigmoidActivation(hiddenLayerOutput)        
#         #Output layer calculations:
#         finalLayerOutput = calculateOutput(hiddenLayerOutput, network[1])
#         finalLayerOutput = sigmoidActivation(finalLayerOutput)
#         #Save the predictions
#         predictions.append(finalLayerOutput)
#     return predictions

# def forwardPropagationNew(input_seeds_list, network):
#     for i in range(len(input_seeds_list)):
#         layerInput = input_seeds_list[i]
#         for j in range(len(network)):
#             layer = network[j]
#             output = calculateOutput(layerInput, layer)
#             layerInput = output

def forwardPropagationSingle(input_seed, network):
    layerInput = input_seed
    for j in range(len(network)):
        layer = network[j]
        output = calculateOutput(layerInput, layer)
        layerInput = output
#     pp.pprint(output)
    return output


            
# testInputSeedsList = [[1,0]]
# forwardPropagationNew(testInputSeedsList, network)
# print(network)
testInputSeed = [1,0]
forwardPropagationSingle(testInputSeed, network)
pp.pprint(network)
# for j in range(len(network)):
#     layer = network[j]
#     for eachNeuron in layer:
#         print(eachNeuron['weights'])
#         break

[[{'output': 0.7594184628136014,
   'weights': [0.5714025946899135, 0.4288890546751146, 0.5780913011344704]}],
 [{'output': 0.7250868186678797,
   'weights': [0.20609823213950174, 0.81332125135732]},
  {'output': 0.8077468842111606,
   'weights': [0.8235888725334455, 0.6534725339011758]}]]


In [13]:
# predictions = forwardPropagation(input_seeds_list,n_hidden=4, n_output=3)
# for i in predictions:
#     pp.pprint(i)
#     break

error = (expected - output_from_forward_propagation) * derivative(output_from_forward_propagation)

In [14]:
# def calculateDerivative(predictions):
#     derivative = []
#     for each in predictions:
#         derivative.append(each * (1 - each))
#     return derivative

# #test
# testPredictions = [0.940540810659955, 0.8623106708953342, 0.9625189765256987]
# testDerivatives = calculateDerivative(testPredictions)
# print(testDerivatives)

In [15]:
# def calculateError(predictions, expected):
#     errors = []
#     derivatives = calculateDerivative(predictions)
#     for i in range(len(predictions)):
#         errors.append((expected[i] - predictions[i]) * derivatives[i])
#     return errors

# #test
# testPredictions = [0.940540810659955, 0.8623106708953342, 0.9625189765256987]
# testExpected = [1,1,1]
# testErrors = calculateError(testPredictions, testExpected)
# print(testErrors)

In [16]:
def sigmoidDerivative(x):
    return (x*(1-x))

In [17]:
# for each in network[1]:
#     print('Here:')
#     pp.pprint(each)
lyr = network[1]
for j in range(len(lyr)):
    print('Here')
    pp.pprint(lyr[j]['output'])
print(expected)

Here
0.7250868186678797
Here
0.8077468842111606
[' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 1', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 2', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3', ' 3

In [18]:
def backPropoagateError(network, expected):
    for i in reversed(range(len(network))):
        error = 0
        layer = network[i]        
        #For the output layer:
        if i == (len(network) - 1):
            for j in range(len(layer)):
                error = expected[j] - layer[j]['output']
                error = error * sigmoidDerivative(error)
                layer[j]['delta'] = error
#                 pp.pprint(str(j)+':'+str(error))
        #for hidden layers:
        else:
            for j in range(len(layer)):
                for eachNeuron in network[i+1]:
                    error += eachNeuron['weights'][j] * eachNeuron['delta']
                error = error * sigmoidDerivative(error)
                layer[j]['delta'] = error

In [19]:
backPropoagateError(network, expected)
pp.pprint(network)

TypeError: unsupported operand type(s) for -: 'str' and 'float'

In [ ]:
# Update network weights with error
def update_weights(network, input_features, l_rate):
    for i in range(len(network)):
        inputs = input_features[:-1]
        print('input '+str(i)+':', inputs)
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']
    

In [ ]:
testInputFeatures = [2.7810836,2.550537003]
testLRate = 0.01
update_weights(network, testInputFeatures, testLRate)
pp.pprint(network)

In [ ]:
for i in range(len(network)):
    print(i)

In [ ]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forwardPropagationSingle(row, network)
            print(outputs)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [ ]:
seed(1)
dataset = [ [2.7810836,2.550537003,0],
            [1.465489372,2.362125076,0],
            [3.396561688,4.400293529,0],
            [1.38807019,1.850220317,0],
            [3.06407232,3.005305973,0],
            [7.627531214,2.759262235,1],
            [5.332441248,2.088626775,1],
            [6.922596716,1.77106367,1],
            [8.675418651,-0.242068655,1],
            [7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = buildNetwork(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)